# fit_generator_flow_from
: 4. fit_generator에서는 flow라는 것으로 이미지를 이미 로드 한 다음에 DataGenerator을 사용했는데
이번 5. fit_generator_flow from에서는 폴더별로 담겨있는 이미지를 한 번에 로드 하는 방법에 대해서 알아보자. 

In [1]:
import os
from glob import glob

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
os.listdir('/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/mnist_png/training/')

['9', '0', '7', '6', '1', '8', '4', '3', '2', '5']

이렇게 폴더별로 labeling 되어 있는 경우는, 담겨있는 이미지는 현재 하는 방법을 이용하는게 매우 유용하다. 

cf) 'for github/인공지능_딥러닝_dataset'에 있는 또 다른 예제인 'cifar'같은 경우, 폴더별이 아니라 파일명에 label이 있기 때문에 이 방법을 쓸 수 없고, 복잡해 진다. 

In [3]:
train_dir = '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/mnist_png/training'
test_dir ='/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/mnist_png/testing/'

## 1. Hyperparameter Tunning

In [4]:
num_epochs = 10
batch_size =32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## 2. Preprocess

In [5]:
train_datagenerator = ImageDataGenerator(
    rescale = 1./255.,
    width_shift_range = 0.3,
    zoom_range = 0.7,
    horizontal_flip = True
)

test_datagenerator = ImageDataGenerator(
    rescale = 1./255.,
)

In [6]:
#train 이미지 generator 생성

#flow시키는 generator 생성
#폴더별로 있으니까
train_generator = train_datagenerator.flow_from_directory(
    train_dir, #앞에서 한 train_dir = '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/mnist_png/training'
    target_size = input_shape[:2], #input_shape = (28, 28, 1)에서 채널빼고 (28, 28)만)
    batch_size = batch_size,
    color_mode = 'grayscale', 
    class_mode = 'categorical'
)
    

Found 60000 images belonging to 10 classes.


In [7]:
#test 이미지 generator 생성
validation_generator = test_datagenerator.flow_from_directory(
    test_dir, #앞에서 한 train_dir = '/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/mnist_png/training'
    target_size = input_shape[:2], #input_shape = (28, 28, 1)에서 채널빼고 (28, 28)만)
    batch_size = batch_size,
    color_mode = 'grayscale', 
    class_mode = 'categorical'
)

Found 10000 images belonging to 10 classes.


## 3. Build Model

In [8]:
#이건 이전에 많이 한 것대로
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, 3, padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = 2)(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, 3, padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = 2)(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net, name = 'Basic_CNN')

In [9]:
#model compile해주기
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate), #Optimization
             loss = 'categorical_crossentropy', #Loss function
             metrics = ['accuracy']) #Metrics/Accuracy

## 4. Training
model.fit_generator(
    generator,
    steps_per_epoch=None,
    epochs=1,
    verbose=1,
    callbacks=None,
    validation_data=None,
    validation_steps=None,
    validation_freq=1,
    class_weight=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    shuffle=True,
    initial_epoch=0,
)

In [ ]:
model.fit_generator(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    epochs = num_epochs,
    validation_data =validation_generator, 
    validation_steps = len(validation_generator))#steps_per_epoch : 한 epoch에 몇 개의 묶음의 이미지(batch)가 epoch에 돌아갈지

Epoch 1/10
